In [2]:
import joblib
import numpy as np
import tensorflow as tf
from PIL import Image
from tensorflow.keras import layers, models
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming your text model is saved as 'text_model.joblib'
text_model = joblib.load('rf_text_sentimodel.joblib')

# Load the image model (CNN) - ensure the file path is correct
image_model = tf.keras.models.load_model('Senti_img.h5')

# Assuming the test set for images is under 'test_dir' and the test images are of grayscale
test_dir = r'C:\Users\AVIGHYAT\OneDrive\Documents\2nd year 1st sem\Statistical ML\datasets\senti_img\test'

# Define a mapping from image classes to text classes
image_class_mapping = {
    'angry': 2,  # Map to Anger in text model
    'disgusted': 2,  # Map to Anger in text model
    'fearful': 3,  # Map to Fear in text model
    'happy': 0,  # Map to Joy in text model
    'neutral': 4,  # Neutral
    'sad': 1,  # Map to Sadness in text model
    'surprised': 5   # Map to Surprise in text model
}


In [3]:
# Function to preprocess images for the image model
def preprocess_image(image_path):
    # Load image from path
    img = Image.open(image_path)
    
    # Resize to 48x48 pixels (as expected by the model)
    img = img.resize((48, 48))
    
    # Convert to grayscale
    img = img.convert('L')
    
    # Convert image to NumPy array and normalize pixel values
    img_array = np.array(img) / 255.0
    
    # Add batch dimension
    img_array = img_array.reshape((1, 48, 48, 1))
    
    return img_array


In [20]:
def predict_sentiment(text, image_path):
    # Load the TfidfVectorizer used to train the text model
    vectorizer = joblib.load('tfidf_vectorizer.joblib')
    
    # Transform text input to match expected input size
    text_vector = vectorizer.transform([text]).toarray()
    
    # Text prediction using the text model
    text_prediction = text_model.predict_proba(text_vector)[0]
    print(f"Text prediction probabilities: {text_prediction}")  # Debugging: Print text prediction
    
    # Image prediction using the image model
    image_features = preprocess_image(image_path)
    image_prediction = image_model.predict(image_features)[0]
    print(f"Image prediction probabilities: {image_prediction}")  # Debugging: Print image prediction
    
    # Map image prediction to the correct class (according to mapping dictionary)
    predicted_class_index = np.argmax(image_prediction)
    image_class = list(image_class_mapping.keys())[predicted_class_index]
    image_class_index = image_class_mapping[image_class]
    print(f"Mapped image class: {image_class}, Index: {image_class_index}")  # Debugging
    
    # Combine text and image predictions using weighting
    weights = [0.5, 0.5]  # Adjust weights as needed
    final_prediction = (weights[0] * text_prediction + weights[1] * image_class_index)
    print(f"Combined prediction: {final_prediction}")  # Debugging
    
    return final_prediction


In [21]:
# Example usage of the predict_sentiment function
text = "This movie was fantastic!"
image_path = r"C:\Users\AVIGHYAT\OneDrive\Documents\2nd year 1st sem\Statistical ML\datasets\senti_img\test\angry\im3.png"

# Predict sentiment
sentiment = predict_sentiment(text, image_path)

# Map numerical sentiment to sentiment labels
sentiment_labels = {0: "Joy", 1: "Sadness", 2: "Anger", 3: "Fear", 4: "Neutral", 5: "Surprise"}

# Print the predicted sentiment
print("Predicted Sentiment:", sentiment_labels[np.argmax(sentiment)])


Text prediction probabilities: [0.01 0.76 0.   0.16 0.07 0.  ]
1/1 [==============================] - 0s 23ms/step
Image prediction probabilities: [8.7834787e-01 1.6241395e-04 1.2091521e-01 6.7586021e-05 1.7928953e-05
 7.5910561e-05 4.1303044e-04]
Mapped image class: angry, Index: 2
Combined prediction: [1.005 1.38  1.    1.08  1.035 1.   ]
Predicted Sentiment: Sadness
